In [1]:
# GLOVE download reference https://github.com/guillaume-chevalier/GloVe-as-a-TensorFlow-Embedding-Layer/blob/master/README.md

import pandas as pd
import numpy as np
import os
import nltk
import re
from datetime import date

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense ,Dropout,LSTM, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.losses import sparse_categorical_crossentropy

from tensorflow.keras.losses import sparse_categorical_crossentropy

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [2]:
tf.__version__

'2.0.0-alpha0'

In [3]:
import chakin

import json

from collections import defaultdict
import zipfile
chakin.search(lang='English')

                   Name  Dimension                     Corpus VocabularySize  \
2          fastText(en)        300                  Wikipedia           2.5M   
11         GloVe.6B.50d         50  Wikipedia+Gigaword 5 (6B)           400K   
12        GloVe.6B.100d        100  Wikipedia+Gigaword 5 (6B)           400K   
13        GloVe.6B.200d        200  Wikipedia+Gigaword 5 (6B)           400K   
14        GloVe.6B.300d        300  Wikipedia+Gigaword 5 (6B)           400K   
15       GloVe.42B.300d        300          Common Crawl(42B)           1.9M   
16      GloVe.840B.300d        300         Common Crawl(840B)           2.2M   
17    GloVe.Twitter.25d         25               Twitter(27B)           1.2M   
18    GloVe.Twitter.50d         50               Twitter(27B)           1.2M   
19   GloVe.Twitter.100d        100               Twitter(27B)           1.2M   
20   GloVe.Twitter.200d        200               Twitter(27B)           1.2M   
21  word2vec.GoogleNews        300      

In [4]:

CHAKIN_INDEX = 13
NUMBER_OF_DIMENSIONS = 25
SUBFOLDER_NAME = "GloVe.6B.200d"

DATA_FOLDER = "embeddings"
ZIP_FILE = os.path.join(DATA_FOLDER, "{}.zip".format(SUBFOLDER_NAME))
ZIP_FILE_ALT = "glove" + ZIP_FILE[5:]  # sometimes it's lowercase only...
UNZIP_FOLDER = os.path.join(DATA_FOLDER, SUBFOLDER_NAME)
GLOVE_FILENAME = os.path.join(UNZIP_FOLDER, "{}.txt".format(SUBFOLDER_NAME))



In [5]:
if not os.path.exists(ZIP_FILE) and not os.path.exists(UNZIP_FOLDER):
    # GloVe by Stanford is licensed Apache 2.0: 
    #     https://github.com/stanfordnlp/GloVe/blob/master/LICENSE
    #     http://nlp.stanford.edu/data/glove.twitter.27B.zip
    #     Copyright 2014 The Board of Trustees of The Leland Stanford Junior University
    print("Downloading embeddings to '{}'".format(ZIP_FILE))
    chakin.download(number=CHAKIN_INDEX, save_dir='./{}'.format(DATA_FOLDER))
else:
    print("Embeddings already downloaded.")


Embeddings already downloaded.


In [6]:
if not os.path.exists(UNZIP_FOLDER):

    with zipfile.ZipFile('embeddings/glove.6B.zip',"r") as zip_ref:
        print("Extracting embeddings to '{}'".format(UNZIP_FOLDER))
        zip_ref.extractall(UNZIP_FOLDER)
else:
    print("Embeddings already extracted.")

Embeddings already extracted.


In [36]:
EMBEDDING_DIM = 300

In [7]:
# generate a class that looks up embeddings


# Not actually necessary , as the keras implementation takes embedded matrix input natively
#class PretrainedEmbedding(tf.keras.layers.Layer):
#    """Embedding layer taking pretrained embeddings"""##

#    def __init__(self, embeddings, rate=0.1, **kwargs):
#        """"Instantiate the layer using a pre-defined embedding matrix."""
#        super().__init__(**kwargs)
#        self.embeddings = tf.constant(embeddings)
#        # if you want to add some dropout (or normalization, etc.)
#        #self.dropout = tf.keras.layers.Dropout(rate=rate)

#    def call(self, inputs, training=None):
#        """Embed some input tokens and optionally apply dropout."""
#        output = tf.nn.embedding_lookup(self.embeddings, inputs)
#        return output #self.dropout(output, training=training)


In [8]:
# list of unwanted characters
badlist = pd.read_csv('data/badlist', header=None)[0].tolist()

In [9]:

# add spaces to certain special characters
unspaced = r'[\[\]\(\)\.\,\/\?\-\!\"\;|:]'
spaced =  ' \g<0> '

In [21]:
def clean_line(line):
    """
    remove stray characters
    """
    for badchar in badlist:
        if badchar in line:
            line = re.sub(badchar,'',line)
            
    line = re.sub(unspaced, spaced, line)  
    line = re.sub('”','',line)
    line = re.sub('’' ,"'", line)
    line = re.sub("'s" ," 's", line)
    line = re.sub("\n" ," \n", line)
    
    return line

In [22]:
# read the raw text files and clean the lines

START_STRING = 'BEGIN EPISODE'

all_episodes_by_sentence = []

for element in os.listdir('house/'):
    if 'clean' in element:
        with open('house/'+element) as in_raw:
            # start token
            all_episodes_by_sentence.append(START_STRING)
            for (i, line) in enumerate(in_raw):
                all_episodes_by_sentence.append(clean_line(line) )
        
            # end token
            all_episodes_by_sentence.append('END EPISODE\n\n')
            all_episodes_by_sentence.append('------------------------------------------\n')
    

In [23]:
def tokenize(sentence):
    """
    simple tokenization keeping the line seperators etc.
    """
    return sentence.lower().split(' ')

In [24]:
# combine text to create a single string for sliceshifting
word_dict = {}
used_words = []
text = []
for sent in all_episodes_by_sentence:
    for word in tokenize(sent):

        if not word in word_dict:
            word_dict[word] = 0
            used_words.append(word)
        word_dict[word] +=1
        text.append(word)
#text = [tokenize(sent) for sent in all_episodes_by_sentence]

In [25]:
# total number of words
len(used_words)

26547

In [26]:
# test if we can reduce the number of words
data_quant = pd.DataFrame.from_dict(word_dict, orient='index').sort_values(by=0, ascending=False)

In [27]:
data_quant.shape

(26547, 1)

In [28]:
data_quant.tail(5000) # looking clean, but many words with only one occurance

,0
transmitter,1
revelations,1
tagline,1
feature,1
grating,1
...,...
reclaimed,1
190,1
clinc,1
seamen,1


In [29]:
# only 7401 words occur more than 6 times
data_quant.loc[data_quant[0] > 6].shape

(7555, 1)

In [30]:
allowed_vocabulary = data_quant.loc[data_quant[0] > 6].index.tolist()

In [31]:
# preformat to test if this is enough
word_dict = {}
used_words = []
text = []
for sent in all_episodes_by_sentence:
    for word in tokenize(sent):
        if word in allowed_vocabulary:
            if not word in word_dict:
                word_dict[word] = 0
                used_words.append(word)
            word_dict[word] +=1
            text.append(word)

In [34]:
# reverse dictionary for word_count
word2idx = {}
idx2word= {}
for idx, word in enumerate(allowed_vocabulary):
    word2idx[word] = idx
    idx2word[idx] =  word 

In [38]:
# load embedding from file
embedded_words = {}
with open('embeddings/GloVe.6B.200d/glove.6B.300d.txt', 'r' ) as glove_file:
    for (i, line) in enumerate(glove_file):
        split = line.split(' ')
            
        word = split[0]
        representation = split[1:]
        representation = np.array([float(val) for val in representation] )
        
        embedded_words[word] = representation

In [39]:
# create an embedding matrix from only the known words
EMBEDDING_MATRIX = np.zeros((len(word2idx.items()) + 1, EMBEDDING_DIM))

for word, vec in embedded_words.items():
    if word in word2idx:
        EMBEDDING_MATRIX[word2idx[word]] = vec

In [41]:
# original text, first 10 sentences
all_episodes_by_sentence[0:10]

['BEGIN EPISODE',
 " [ Open on a House 's face .  His eyes are closed .  The picture is not quite in color ,  but it 's not black and white either .  Radiohead 's No Surprises plays .  ]  \n",
 " [ House opens his eyes .  He 's lying on a twin bed on the left side of a cell - like room at Mayfield .  He 's wearing a gray t - shirt .  There is a stainless steel basin on the tiled floor near his head .  On the opposite wall there is a single window .  Next to it a metal sink is bolted into the wall .  Another stainless basin is on the floor below the sink .  ]  \n",
 " [ Cut to House opening his eyes .  The color has returned to normal .  Everything is quiet .  He isn't restrained any longer .  He touches his thigh briefly then sits up .  He limps to the window ,  holding his leg for support .  ]  \n",
 ' [ Cut to a suitcase dropping on the bed .  House packs his clothes and zips the valise .  ]  \n',
 ' [ Cut to House walking down the hall with his cane in his right hand and his suitcas

In [40]:
' '.join(text[0:200]) # missing a few proper nouns like Radiohead, but this should do

"begin episode  [ open on a house 's face .  his eyes are closed .  the picture is not quite in color ,  but it 's not black and white either .  's no surprises plays .  ]  \n  [ house opens his eyes .  he 's lying on a twin bed on the left side of a cell - like room at mayfield .  he 's wearing a gray t - shirt .  there is a steel basin on the floor near his head .  on the opposite wall there is a single window .  next to it a metal sink is into the wall .  another basin is on the floor below the sink .  ]  \n  [ cut to house opening his eyes .  the color has returned to normal .  everything is quiet .  he isn't any longer .  he touches his thigh briefly then sits up .  he limps to the window ,  holding his leg for support .  ]  \n  [ cut to a suitcase dropping on the"

In [42]:
# The unique characters in the file
vocab = sorted(allowed_vocabulary)
print ('{} unique characters'.format(len(vocab)))

7555 unique characters


In [29]:
# Creating a mapping from unique characters to indices
# Here we are acutally using the pretrained words, so this is not needed
#word2idx = {u:i for i, u in enumerate(vocab)}
#idx2word = np.array(vocab)

In [43]:
text_as_int = np.array([word2idx[c] for c in text])
text_as_int

array([675, 714,   0, ..., 280, 821, 820])

In [46]:
# The maximum length sentence we want for a single input in words
seq_length = 20  # 20 words traininsdata
examples_per_epoch = len(text)//(seq_length+1)

In [47]:
text_as_int

array([675, 714,   0, ..., 280, 821, 820])

In [48]:
# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [49]:
# extract sequences from character dataset
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

In [50]:

def split_shift_input(segment):
    """
    Creates the teaching data by shifting the training data on off to create labeled data
    """
    input_segment = segment[:-1]
    target_segment = segment[1:]
    return input_segment, target_segment

dataset = sequences.map(split_shift_input)

In [51]:
# set up dataset as prebatched
BATCH_SIZE = 20

# shuffle the dataset and set batch size
dataset = dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder=True)

In [52]:


# Length of the vocabulary
vocab_size = len(vocab)

# hyperparameters

# embedding dimension
embedding_dim = 300

# RNN units
rnn_units = 850


In [63]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size,  dropout = False):
    
    i = Input(shape=(None,), batch_size=batch_size, dtype=tf.int64 )
    # use pretrained embedding now
    
    # feed the embedding matrix into the Embedding layer, set trainable to False
    x = Embedding(vocab_size, embedding_dim,weights=EMBEDDING_MATRIX,  trainable=False)(i)
    
    if dropout:
        dropout = 0.2
    else:
        dropout = 0.0
    
    x = LSTM(rnn_units, 
             return_sequences=True,
             dropout=dropout,
             stateful=True)(x)
    
    x = Dense(vocab_size)(x)
    
    model = Model(i,x)
    
    return model
    

In [64]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [65]:
# simple model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(20, None)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (20, None, 300)           2266500   
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (20, None, 850)           3913400   
_________________________________________________________________
dense_1 (Dense)              (20, None, 7555)          6429305   
Total params: 12,609,205
Trainable params: 10,342,705
Non-trainable params: 2,266,500
_________________________________________________________________


In [66]:
def loss(labels, logits):
    """
    Define loss function 
    """
    return sparse_categorical_crossentropy(labels, logits, from_logits=True)


In [67]:
model.compile(optimizer='adam', loss =loss)# loss='sparse_categorical_crossentropy' )

In [68]:
# Directory where the checkpoints will be saved
today = date.today()

checkpoint_dir = './pretrained_word_training_checkpoints_{today}'.format(today=today)


checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# define callbacks
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [69]:
EPOCHS=50

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
4501/4501 [==============================] - 766s 170ms/step - loss: 4.0311
Epoch 2/50
1914/4501 [===========>..................] - ETA: 7:18 - loss: 3.6434

In [76]:
# for prediction, batch size has to be changed
# So reload the model and set shape to [1, None]

tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1, embedding_matrix= embedding_matrix)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]),)

model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(1, None)]               0         
_________________________________________________________________
pretrained_embedding_6 (Pret (1, None, 300)            0         
_________________________________________________________________
unified_lstm_6 (UnifiedLSTM) (1, None, 1000)           5204000   
_________________________________________________________________
dense_12 (Dense)             (1, None, 7401)           7408401   
_________________________________________________________________
dense_13 (Dense)             (1, None, 7401)           54782202  
Total params: 67,394,603
Trainable params: 67,394,603
Non-trainable params: 0
_________________________________________________________________


In [77]:
def text_gen(model, start_string=START_STRING, freedom=1.0, num_generate=1000):
    """
    generate text with the trained model
    
    start_string (STR):  Basis for the model to start prediction on. 
    freedom (FLOAT): Multiplier for predictions. The lower it is the lower the impact of predictive variance
    num_generate (INT): Desired text length
    """
    
    text_generated = []
    

    # vectorization of starting string
    input_eval = [word2idx[s] for s in tokenize(start_string)]
    input_eval = tf.expand_dims(input_eval, 0)

    model.reset_states()
    
    
    for i in range(num_generate):
        predictions = model(input_eval)
        
        
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / freedom
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(' ') # spacing
        text_generated.append(idx2word[predicted_id])

    return (start_string + ''.join(text_generated))

In [79]:
print(text_gen(model, start_string=u"BEGIN EPISODE", freedom=1, num_generate=10))

BEGIN EPISODE the the the the the the the the the the
